In [1]:
import math
import re as re
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve

In [2]:
# Get the data
r = requests.get("https://www.hltv.org/stats/teams")

# Create a pandas with pulled data
root = bs(r.content, "html.parser")
root.prettify()
table = (str)(root.find("table"))
teams = pd.read_html(table, header=0)[0]

# Pull the country data out of the table and put into our dataframe
table = table.strip("\\n")
countries = (re.findall('title="(.+)"/>', table))
teams['Country'] = pd.Series(countries)

# Clean up columns
teams.columns = ['Team', 'Maps', 'K/D Differential', 'K/D', 'Rating', 'Country']
teams = teams[['Team', 'Country', 'Maps', 'K/D Differential', 'K/D', 'Rating']]

# Delete Tyloo from dataframe for reasons mentioned above
teams = teams.drop([0])

# Get a series with all team names
teamlist = teams[teams.columns[0]]

# Display resultant panda
teams

,Team,Country,Maps,K/D Differential,K/D,Rating
1,Renegades,Australia,1066,4642,1.05,1.05
2,ViCi,China,731,5761,1.09,1.05
3,ORDER,Australia,873,9638,1.14,1.05
4,Vitality,France,798,3496,1.05,1.05
5,Space Soldiers,Turkey,804,6008,1.09,1.04
6,MIBR,Brazil,775,2993,1.05,1.04
7,FURIA,Brazil,959,6839,1.09,1.04
8,Liquid,United States,1605,8498,1.06,1.04
9,Astralis,Denmark,1319,10481,1.10,1.04
10,FaZe,Europe,1309,5151,1.05,1.04


In [13]:
# Get the data
r = requests.get("https://www.hltv.org/stats/players")

# Create a pandas with pulled data
root = bs(r.content, "html.parser")
root.prettify()

# Pull the player data out of the table and put into our dataframe
table = str(root.find("table"))
players = pd.read_html(table, header=0)[0]
print(players)
# An array containing a list of top 10 teams (according to HLTV's critically acclaimed ranking system)
topTeams = ['Renegades', 'ViCi', 'ORDER', 'Vitality', 'Space Soldiers', 'MIBR', 'FURIA', 'Liquid', 'Astralis', 'FaZe']

# Discard player information for anyone not listed in the top 10 teams
players = players[players['Teams'].isin(topTeams)]

players = players.sort_values('Teams')
players.columns = ['Player', 'Teams', 'Maps','Rounds', 'K/D Differential', 'K/D', 'Rating']

# Display resultant panda
players

      Player  Teams  Maps  Rounds  K-D Diff   K/D  Rating1.0
0      ZywOo    NaN  1017   26721      6149  1.37       1.27
1     s1mple    NaN  1561   41205      9048  1.34       1.25
2      sh1ro    NaN   880   23348      5592  1.45       1.23
3       deko    NaN   404   10913      2341  1.37       1.21
4     saffee    NaN   397   10498      1939  1.30       1.20
..       ...    ...   ...     ...       ...   ...        ...
801    B1ad3    NaN   899   23232     -3408  0.80       0.83
802  netrick    NaN   455   11976     -1736  0.80       0.82
803   HUNDEN    NaN  1578   41153     -6238  0.78       0.81
804   RuFire    NaN   384   10185     -1927  0.74       0.78
805    OCEAN    NaN   410   10602     -1962  0.74       0.77

[806 rows x 7 columns]


,Player,Teams,Maps,Rounds,K/D Differential,K/D,Rating
